In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)

In [2]:
df1=pd.read_csv("bengaluru_house_prices.csv")
df1.head()

FileNotFoundError: [Errno 2] No such file or directory: 'bengaluru_house_prices.csv'

In [ ]:
df1.groupby('area_type')['area_type'].agg('count')

In [ ]:
df2=df1.drop(['area_type','society','balcony','availability'],axis='columns')
df2.head()

In [ ]:

df3=df2.dropna()
df2.isnull().sum()

In [ ]:
df3.isnull().sum()

In [ ]:
df3['size'].unique()

In [ ]:
df3['bhk']=df3['size'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
df3.head()

In [ ]:
df3['bhk'].unique()

In [ ]:
df3['total_sqft'].unique()

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False;
    return True;


In [ ]:
df3[~df3['total_sqft'].apply(is_float)]

In [ ]:
def covt(x):
    token=x.split('-')
    if len(token)==2:
        return ((float(token[0])+float(token[1]))/2)
    try:
        return float(x)
    except:
        return None


In [ ]:
df4=df3.copy()
df4['total_sqft']=df4['total_sqft'].apply(covt)

In [ ]:
df4['total_sqft']

In [ ]:
df4.loc[30]

In [ ]:
df5=df4.copy()
df5['price_sqft']=df5['price']*1000/df5['total_sqft']

In [ ]:
df5.head()

In [ ]:
df5['price']=df5['price']*1000

In [ ]:
location_stats=df5.groupby('location')['location'].agg('count').sort_values()

In [ ]:
location_stats

In [ ]:
other =location_stats[location_stats<=10]

In [ ]:
other

In [ ]:
location_stats

In [ ]:
df5.location=df5.location.apply(lambda x: 'other' if x in other else x)

In [ ]:
df5['location'].head(100)

In [ ]:
df5['location']=df4['location']

In [ ]:
df5.location=df5.location.apply(lambda x:'other' if x in other else x)

In [ ]:
df5

In [ ]:
df5.count().unique()

In [ ]:
len(df5.location.unique())

In [ ]:
def custm(x):
    token=x.split(' ')
    if len(token)==2:
        return token[0]
    try:
        return int(token)
    except:
        return int(x)

In [ ]:

df5.size

In [ ]:
df5['size']=df5['size'].apply(custm)

In [ ]:
df5

In [ ]:
df6=df5.copy()

In [ ]:
df6

In [ ]:
df6.hist()

In [ ]:
plt.hist(df6['price_sqft'],rwidth=0.8)
plt.xlabel('Price_Per_Sqft')
plt.ylabel('count')

In [ ]:
df5.head()

In [ ]:
dummy=pd.get_dummies(df5['location'])


In [ ]:
dummy.drop('other',axis='columns')

In [ ]:
df5=df5.drop('location',axis='columns')

In [ ]:
df5.join(dummy)


In [ ]:
df4

In [ ]:
x=df5.drop(['price'],axis='columns')

In [ ]:
y=df5.price

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=10)

In [ ]:
df4.isnull().sum()

In [ ]:
df5=df4.dropna()

In [ ]:
df5.isnull().sum()

In [ ]:
df5

In [ ]:
df5=df5.drop(['size'],axis='columns')

In [ ]:
df5

In [ ]:
df6=df5[~(df5.total_sqft/df5.bhk<300)]

In [ ]:
df6

In [ ]:
dummy=pd.get_dummies(df6.location)

In [ ]:
df6.join(dummy)

In [ ]:
df7=df6.drop(['location'],axis='columns')

In [ ]:
df7

In [ ]:
df6

In [ ]:
df8=pd.concat([df7,dummy],axis='columns')

In [ ]:
df8

In [ ]:
x=df8.drop('price',axis='columns')

In [ ]:
y=df8.price

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf=LinearRegression()
lr_clf.fit(x_train,y_train)
lr_clf.score(x_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
cross_val_score(LinearRegression(),x,y,cv=cv)

In [ ]:
def predict_price(location,sqft,bath,bhk):
    loc_index=np.where(x.columns==location)[0][0]
    X=np.zeros(len(x.columns))
    X[0]=sqft
    X[1]=bath
    X[2]=bhk
    if loc_index>=0:
        X[loc_index]=1
    return lr_clf.predict([X])[0]

In [ ]:
predict_price('1st Phase JP Nagar',1000,2,2)

In [ ]:
predict_price('Indra Nagar',5000,5,5)

In [ ]:
import pickle
with open('bengaluru_house_prices_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)


In [ ]:
import json
columns={ 'data_columns': [col.lower() for col  in x.columns]}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))